In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold

In [2]:
# load experimental data into pandas dataframe

df_exp = pd.read_csv("team-a.csv")
df_exp = df_exp.drop(['formula'],axis=1)

In [23]:
# split experimental data

X_exp = df_exp[['MagpieData maximum MendeleevNumber', 'MagpieData mean AtomicWeight',
       'MagpieData minimum MeltingT', 'MagpieData maximum MeltingT',
       'MagpieData mean MeltingT', 'MagpieData minimum Column',
       'MagpieData range Column', 'MagpieData avg_dev Column',
       'MagpieData mode Column', 'MagpieData range Row', 'MagpieData mean Row',
       'MagpieData range Electronegativity',
       'MagpieData avg_dev Electronegativity',
       'MagpieData mode Electronegativity', 'MagpieData mean NpValence',
       'MagpieData maximum NdValence', 'MagpieData range NdValence',
       'MagpieData mean NdValence', 'MagpieData maximum NfValence',
       'MagpieData mean NfValence', 'MagpieData mean NValence',
       'MagpieData mode NValence', 'MagpieData maximum NpUnfilled',
       'MagpieData range NpUnfilled', 'MagpieData mean NpUnfilled',
       'MagpieData range NUnfilled', 'MagpieData mean NUnfilled',
       'MagpieData mode NUnfilled', 'MagpieData minimum GSvolume_pa',
       'MagpieData mode GSvolume_pa', 'MagpieData maximum GSbandgap',
       'MagpieData range GSbandgap', 'MagpieData mode GSbandgap',
       'MagpieData mean GSmagmom', 'MagpieData mode SpaceGroupNumber']].values


y_exp = df_exp['gap expt'].values
y_exp = y_exp.reshape(-1,1)
X_train_exp,X_val_exp_holdout,y_train_exp,y_val_exp_holdout = train_test_split(X_exp,y_exp,test_size=0.2,random_state=42)

In [ ]:
#load dft bandgap dataset

df_dft = pd.read_csv("dft_bandgap_dataset.csv")

In [11]:
# extract important features based on importance in xgboost model and load data

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

X_dft = df_dft[['MagpieData maximum MendeleevNumber', 'MagpieData mean AtomicWeight',
       'MagpieData minimum MeltingT', 'MagpieData maximum MeltingT',
       'MagpieData mean MeltingT', 'MagpieData minimum Column',
       'MagpieData range Column', 'MagpieData avg_dev Column',
       'MagpieData mode Column', 'MagpieData range Row', 'MagpieData mean Row',
       'MagpieData range Electronegativity',
       'MagpieData avg_dev Electronegativity',
       'MagpieData mode Electronegativity', 'MagpieData mean NpValence',
       'MagpieData maximum NdValence', 'MagpieData range NdValence',
       'MagpieData mean NdValence', 'MagpieData maximum NfValence',
       'MagpieData mean NfValence', 'MagpieData mean NValence',
       'MagpieData mode NValence', 'MagpieData maximum NpUnfilled',
       'MagpieData range NpUnfilled', 'MagpieData mean NpUnfilled',
       'MagpieData range NUnfilled', 'MagpieData mean NUnfilled',
       'MagpieData mode NUnfilled', 'MagpieData minimum GSvolume_pa',
       'MagpieData mode GSvolume_pa', 'MagpieData maximum GSbandgap',
       'MagpieData range GSbandgap', 'MagpieData mode GSbandgap',
       'MagpieData mean GSmagmom', 'MagpieData mode SpaceGroupNumber']].values
y_dft = df_dft['gap pbe'].values
y_dft = y_dft.reshape(-1,1)

X_train_dft,X_val_dft_holdout,y_train_dft,y_val_dft_holdout = train_test_split(X_dft,y_dft,test_size=0.2,random_state=42)

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)

        return out

In [ ]:
# set up k-fold cross validation

k = 3
kf = KFold(n_splits=k,shuffle=True)

In [19]:
# initialise and train model
# key point here - need to split data, scale, covert to tensors, and load for each fold for cross validation, then train model. 

input_size = X_train_dft.shape[1]
hidden_size = 128
num_classes = 1

val_losses = []

for train_index, val_index in kf.split(X_train_dft):
    X_train, X_val = X_train_dft[train_index], X_train_dft[val_index]
    y_train, y_val = y_train_dft[train_index], y_train_dft[val_index]

    # scale data. be careful, need seperate x scaler and y scaler because they are different shapes
    # also watch out for fit_transform for the train data and .transform for test. fit_transform calculates mean/std from data as well as tranformation 
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1,1))
    X_val_holdout_scaled = scaler_X.transform(X_val_dft_holdout)
    y_val_holdout_scaled = scaler_y.transform(y_val_dft_holdout.reshape(-1,1))

    # convert to tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled)
    y_train_tensor = torch.FloatTensor(y_train_scaled)
    X_val_tensor = torch.FloatTensor(X_val_holdout_scaled)
    y_val_tensor = torch.FloatTensor(y_val_holdout_scaled)

    # create dataloader - this feeds data in batches during training 
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor),batch_size=32,shuffle=True)

    # create model
    model = SimpleNet(input_size, hidden_size, num_classes)

    # use different criterion for classification problems. this optimiser is standard, can change lr if results are poor
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # training loop
    num_epochs = 50
    for epoch in range(num_epochs):
        # proper implementation of dataloader:
        for batch_X, batch_y in train_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # evaluation - calculates accuracies and adds to list initialised at start
    with torch.no_grad():
        outputs = model(X_val_tensor)
        outputs_original = scaler_y.inverse_transform(outputs.numpy())
        y_val_original = scaler_y.inverse_transform(y_val_tensor.numpy())

        val_loss = np.mean(np.abs(outputs_original -y_val_original))
        val_losses.append(val_loss)
        print(f"fold validation MAE:{val_loss.item()}")
            
# calculates accuracy
avg_val_loss = np.mean(val_losses)
print(f"MAE:{avg_val_loss}")
print(f"mae standard deviation:{np.std(val_losses)}")

fold validation MAE:0.4490479826927185
fold validation MAE:0.4519434869289398
fold validation MAE:0.447721391916275
fold validation MAE:0.4412199854850769
fold validation MAE:0.4541642665863037
MAE:0.4488193988800049
mae standard deviation:0.004412176087498665


In [25]:
# now to fine-tune model trained on dft bandgap to work on experimental bandgap:

# freeze layers of pretrained NN:

# Freeze all except the last layer
for i, p in enumerate(model.parameters()):
    p.requires_grad = True  # start by enabling all
# If your last layer is model[-1], freeze earlier layers:
for p in list(model.parameters())[:-2]:  # heuristic: freeze everything except last Linear weights+bias
    p.requires_grad = False

# train on dataset:

input_size = X_train_exp.shape[1]
hidden_size = 128
num_classes = 1

val_losses = []

for train_index, val_index in kf.split(X_train_exp):
    X_train, X_val = X_train_exp[train_index], X_train_exp[val_index]
    y_train, y_val = y_train_exp[train_index], y_train_exp[val_index]

    # scale data. be careful, need seperate x scaler and y scaler because they are different shapes
    # also watch out for fit_transform for the train data and .transform for test. fit_transform calculates mean/std from data as well as tranformation 
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1,1))
    X_val_holdout_scaled = scaler_X.transform(X_val_exp_holdout)
    y_val_holdout_scaled = scaler_y.transform(y_val_exp_holdout.reshape(-1,1))

    # convert to tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled)
    y_train_tensor = torch.FloatTensor(y_train_scaled)
    X_val_tensor = torch.FloatTensor(X_val_holdout_scaled)
    y_val_tensor = torch.FloatTensor(y_val_holdout_scaled)

    # create dataloader - this feeds data in batches during training 
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor),batch_size=32,shuffle=True)

    # create model
    model = SimpleNet(input_size, hidden_size, num_classes)

    # crucial - use different optimiser with much lower learning rate
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3, weight_decay=1e-4)

    # training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        for batch_X, batch_y in train_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # evaluation - calculates accuracies and adds to list initialised at start
    with torch.no_grad():
        outputs = model(X_val_tensor)
        outputs_original = scaler_y.inverse_transform(outputs.numpy())
        y_val_original = scaler_y.inverse_transform(y_val_tensor.numpy())

        val_loss = np.mean(np.abs(outputs_original -y_val_original))
        val_losses.append(val_loss)
        print(f"fold validation MAE:{val_loss.item()}")
            
# calculates accuracy
avg_val_loss = np.mean(val_losses)
print(f"MAE:{avg_val_loss}")
print(f"mae standard deviation:{np.std(val_losses)}")

fold validation MAE:0.4685835838317871
fold validation MAE:0.42799195647239685
fold validation MAE:0.44363340735435486
fold validation MAE:0.44315817952156067
fold validation MAE:0.4302848279476166
MAE:0.4427303671836853
mae standard deviation:0.014432597905397415


In [ ]:
# good MAE for a first attempt and proves concept that transfer learning could work for these datasets. 
# ideas for next steps:
# - explore the dft bandgap dataset a bit more, does it contain a load of organics that are not relevent for our inorganic dataset for example?
# - try fine-tuning the fine-tuning - changing no. of epochs to '100' and increasing learning rate has just returned a better model than the NN trained only on bandgap data!! great result
# - have a look at if there is something wrong with the model, since it is better at using dft bandgap with fine-tuning on experimental bandgap, than it is at predicting dft bandgap 